In [1]:
import json
import random

In [7]:
def process_jsonl(file_path):
    content_count = {}

    with open(file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            for message in entry['messages']:
                if message['role'] == 'assistant':
                    content = message['content']
                    if content in content_count:
                        content_count[content] += 1
                    else:
                        content_count[content] = 1

    return content_count

In [4]:
drug_count = process_jsonl('final_finetune_dataset.jsonl')

In [7]:
len(drug_count)

870

In [5]:
# maximal number of appearces of a single drug in different prompts
max = 0
for count in drug_count.values():
    if count > max:
        max = count
print(max)

97


In [4]:
def split_jsonl_for_training_and_test(file_path, num_train, num_test):
    # Read data from the original file
    with open(file_path, 'r') as file:
        all_data = [json.loads(line) for line in file]

    if len(all_data) < num_train + num_test:
        raise ValueError("The file does not contain enough entries for the specified training and test set sizes.")

    # Randomly select entries for the training set
    train_data = random.sample(all_data, num_train)
    remaining_data = [entry for entry in all_data if entry not in train_data]

    # Extract "assistant: content" from training set
    train_contents = {message['content'] for entry in train_data for message in entry['messages'] if message['role'] == 'assistant'}

    # Select entries for the test set ensuring no overlap in "assistant: content"
    test_data = []
    attempts = 0
    while len(test_data) < num_test and attempts < 10:
        candidate = random.choice(remaining_data)
        candidate_contents = {message['content'] for message in candidate['messages'] if message['role'] == 'assistant'}
        if not train_contents & candidate_contents:
            test_data.append(candidate)
            remaining_data.remove(candidate)
        else:
            attempts += 1

    if len(test_data) < num_test:
        return "Error: Unable to create a test set without overlap after 10 attempts."

    # Write training and test sets to new files
    with open(f'experiments/training_set_{num_train}.jsonl', 'w') as train_file:
        for entry in train_data:
            train_file.write(json.dumps(entry) + '\n')

    with open(f'experiments/test_set_{num_test}.jsonl', 'w') as test_file:
        for entry in test_data:
            test_file.write(json.dumps(entry) + '\n')

    return 'Training and test sets created successfully.'

In [6]:
split_jsonl_for_training_and_test('final_finetune_dataset.jsonl', 10, 10)

'Training and test sets created successfully.'

In [8]:
process_jsonl('experiments/training_set_10.jsonl')

{'CN(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C(=O)NC(CCC(=O)[O-])C(=O)[O-].[Na+].[Na+]': 1,
 'CCC1C(C(C(N(CC(CC(C(C(C(C(C(=O)O1)C)OC2CC(C(C(O2)C)O)(C)OC)C)OC3C(C(CC(O3)C)N(C)C)O)(C)O)C)C)C)O)(C)O': 1,
 'CC1=NC=C(C(=C1O)CO)CO.Cl': 1,
 'CC(C)C(CCCN(C)CCC1=CC(=C(C=C1)OC)OC)(C#N)C2=CC(=C(C=C2)OC)OC.Cl': 1,
 'C1CCN(CC1)C2=NC(=NC3=C2N=C(N=C3N4CCCCC4)N(CCO)CCO)N(CCO)CCO': 1,
 'CC(C)C1=C(C(=C(N1CCC(CC(CC(=O)[O-])O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4.CC(C)C1=C(C(=C(N1CCC(CC(CC(=O)[O-])O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4.[Ca+2]': 1,
 'CNS(=O)(=O)CC1=CC2=C(C=C1)NC=C2CCN(C)C.C(CC(=O)O)C(=O)O': 1,
 'CC(C)(C(=O)[O-])OC1=CC=C(C=C1)C(=O)C2=CC=C(C=C2)Cl.C[N+](C)(C)CCO': 1,
 'C1C(=NC(=O)N1N=CC2=CC=C(O2)C3=CC=C(C=C3)[N+](=O)[O-])[O-].[Na+]': 1,
 'CCN1CC(C(C1=O)(C2=CC=CC=C2)C3=CC=CC=C3)CCN4CCOCC4.Cl': 1}

In [9]:
process_jsonl('experiments/test_set_10.jsonl')

{'CC(=O)O.C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)N': 1,
 'C1=CC(=C(C=C1Cl)Cl)COC(CN2C=CN=C2)C3=C(C=C(C=C3)Cl)Cl': 1,
 'CC(C)(C)C(C=CC1=CC2=C(C=C1)OCO2)O': 1,
 'CC1(OC2CC3C4CC(C5=CC(=O)C=CC5(C4(C(CC3(C2(O1)C(=O)CO)C)O)F)C)F)C': 1,
 '[Cu]': 1,
 'COC1C(C(C(C(O1)COS(=O)(=O)[O-])OC2C(C(C(C(O2)C(=O)[O-])OC3C(C(C(C(O3)COS(=O)(=O)[O-])OC4C(C(C(C(O4)C(=O)[O-])OC5C(C(C(C(O5)COS(=O)(=O)[O-])O)O)NS(=O)(=O)[O-])O)O)OS(=O)(=O)[O-])NS(=O)(=O)[O-])O)OS(=O)(=O)[O-])O)NS(=O)(=O)[O-].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+]': 1,
 'C1=CC(=C(C=C1F)F)C(CN2C=NC=N2)(CN3C=NC=N3)O': 1,
 'CC(C)OC1=C(C=C(C=C1)C2=NC(=NO2)C3=C4CCC(C4=CC=C3)NCCO)C#N.Cl': 1,
 'CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)C(C(C5=CC=CC=C5)NC(=O)OC(C)(C)C)O)O)OC(=O)C6=CC=CC=C6)(CO4)OC(=O)C)O)C)O': 1,
 'CC(C(C1=CC=CC=C1)O)NC.Cl': 1}